Verified in the shell that both files ran through the python interpreter without errors.

In [1]:
from cpso_particle import COMB_Particle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import expit
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
part1 = COMB_Particle(2.1, 2.1, 2.1, 5, (-6.0, 6.0), (-4.0, 0.25), (0.4, 0.9))

I need to reorder the lines in the `__init__` method so that all the static attributes are assigned before any other methods are called.

In [3]:
dir(part1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'b',
 'c1',
 'c2',
 'c3',
 'convert_pos_to_binary',
 'initialize_position',
 'initialize_velocity',
 'jaccard_index',
 'ndim',
 'p_fitness',
 'pbest',
 'pbinary',
 'update_binary_position',
 'update_inertia',
 'update_position',
 'update_velocity',
 'v',
 'v_bounds',
 'w',
 'w_bounds',
 'x',
 'x_bounds']

In [4]:
print('{:10}: {}'.format('b', part1.b))
print('{:10}: {}'.format('c1', part1.c1))
print('{:10}: {}'.format('c2', part1.c2))
print('{:10}: {}'.format('c3', part1.c3))
print('{:10}: {}'.format('ndim', part1.ndim))
print('{:10}: {}'.format('p_fitness', part1.p_fitness))
print('{:10}: {}'.format('pbest', part1.pbest))
print('{:10}: {}'.format('pbinary', part1.pbinary))
print('{:10}: {}'.format('v', part1.v))
print('{:10}: {}'.format('v_bounds', part1.v_bounds))
print('{:10}: {}'.format('w', part1.w))
print('{:10}: {}'.format('w_bounds', part1.w_bounds))
print('{:10}: {}'.format('x', part1.x))
print('{:10}: {}'.format('x_bounds', part1.x_bounds))

b         : [False False False  True False]
c1        : 2.1
c2        : 2.1
c3        : 2.1
ndim      : 5
p_fitness : 0.0
pbest     : [-0.6436683  -4.60231346  0.06766747  5.2844454   0.04817905]
pbinary   : [False False False  True False]
v         : [-3.9338973  -0.92535269 -1.32961543 -1.59761497 -3.43179165]
v_bounds  : (-4.0, 0.25)
w         : 0
w_bounds  : (0.4, 0.9)
x         : [-0.6436683  -4.60231346  0.06766747  5.2844454   0.04817905]
x_bounds  : (-6.0, 6.0)


This all looks pretty much like I'd expect. A few notes:
* pbest and pbinary should be copies of x and b, that's good.
* p_fitness and w are initialized manually by the swarm, so they still have their zero placeholders

Things to follow up on:
* I don't know if c1, c2, and c3 initialized properly because they're all the same value, I should run another initialization with all different values just to be sure.
* v should NOT be all zeros. Something went wrong with `initialize_velocity()`.
* I don't know if the values for the binary and position vectors are correct because I haven't checked those methods yet. However, everything looks to have the correct type and shape.

**FIX:** `initialize_velocity()` was setting `v` instead of `self.v`.

In [5]:
throwaway = COMB_Particle(2.1, 2.2, 2.3, 5, (-6.0, 6.0), (-4.0, 0.25), (0.4, 0.9))
print('{:10}: {}'.format('b', throwaway.b))
print('{:10}: {}'.format('c1', throwaway.c1))
print('{:10}: {}'.format('c2', throwaway.c2))
print('{:10}: {}'.format('c3', throwaway.c3))
print('{:10}: {}'.format('ndim', throwaway.ndim))
print('{:10}: {}'.format('p_fitness', throwaway.p_fitness))
print('{:10}: {}'.format('pbest', throwaway.pbest))
print('{:10}: {}'.format('pbinary', throwaway.pbinary))
print('{:10}: {}'.format('v', throwaway.v))
print('{:10}: {}'.format('v_bounds', throwaway.v_bounds))
print('{:10}: {}'.format('w', throwaway.w))
print('{:10}: {}'.format('w_bounds', throwaway.w_bounds))
print('{:10}: {}'.format('x', throwaway.x))
print('{:10}: {}'.format('x_bounds', throwaway.x_bounds))

b         : [False False False  True  True]
c1        : 2.1
c2        : 2.2
c3        : 2.3
ndim      : 5
p_fitness : 0.0
pbest     : [-5.27198033 -1.52905092  1.30228432 -0.25221179  3.50845514]
pbinary   : [False False False  True  True]
v         : [-1.850354   -1.97529278 -1.16501075 -3.93817826 -2.1103038 ]
v_bounds  : (-4.0, 0.25)
w         : 0
w_bounds  : (0.4, 0.9)
x         : [-5.27198033 -1.52905092  1.30228432 -0.25221179  3.50845514]
x_bounds  : (-6.0, 6.0)


This looks good. The places where there should be randomness are different from each other between part1 and the throwaway. Additionally, none of the values of x or v are outside the bounds. v is fixed; the c constants populated correctly. The `__init__` function appears to be doing what I want.

Let's check on the `initialize_position` and `initialize_velocity` functions

According to the documentation, numpy.random.uniform returns an nd array of values drawn from a uniform distribution over the interval \[low, high\). Each of these methods *should* give back a 1-Dimensional ndarray of random values in the correct bounds. Let me sanity check a few things.

In [6]:
part1.initialize_position()

In [7]:
type(part1.x)

numpy.ndarray

In [8]:
part1.x.shape

(5,)

In [9]:
part1.ndim

5

In [10]:
part1.x

array([-0.47056802,  5.58439167, -4.34797764, -0.99226722,  5.25983447])

In [11]:
for i in range(10):
    part1.initialize_position()
    print(part1.x)

[ 1.51322349 -1.31315356 -1.48935144  0.35901691 -2.639056  ]
[-5.89948664 -0.70492015 -4.50551678 -0.50769804 -5.30104814]
[4.91815469 2.12233929 5.06730288 3.8929764  2.20704014]
[ 4.11659714  0.76765189  5.84012724 -2.76624241  3.47432598]
[ 3.82506759  0.70558956 -4.64462629  2.89442318 -5.79399248]
[-5.94149496  3.99879613  3.61893008 -5.86201252  0.50337249]
[ 3.80994756 -5.24349006 -2.5283742   5.79571998 -2.61611503]
[ 0.7452442  -2.21763309  1.32603986 -0.35210195  5.81649206]
[-5.49046921  3.32601708  4.65353145  4.90415209 -2.56487149]
[-2.04763478  1.82113745 -1.56665592  5.87793031  2.20207426]


In [12]:
part1.x_bounds

(-6.0, 6.0)

part1.x has the correct type and shape after calling `initialize_position()`. It looks good to the eye. Calling it 10 times in a row appears to give randomness (it certainly doesn't repeat) and none of the 100 values go outside the bounds. I don't think it's necessary to check that numpy.random.uniform gives a uniform distribution of values.

In [13]:
part1.initialize_velocity()

In [14]:
type(part1.v)

numpy.ndarray

In [15]:
part1.v.shape

(5,)

In [16]:
part1.ndim

5

In [17]:
part1.v

array([-1.447529  , -1.0042817 , -3.35079024, -0.79029877, -0.42973469])

In [18]:
for i in range(10):
    part1.initialize_velocity()
    print(part1.v)

[-1.58463188 -3.85911867 -1.7701193  -2.3253065  -2.3406663 ]
[-3.82816851 -0.54636607 -1.74834598 -3.21807183 -1.03098214]
[-2.21079059 -3.20218952 -3.95163175 -3.80371644 -3.63523202]
[-3.13406972 -0.00709941 -2.87474132 -1.82160361 -2.09921244]
[-0.751557   -2.30317622 -0.31881909 -3.50968108 -0.20630922]
[-1.82285285 -0.3712144  -3.03610235 -2.17164294 -1.2417643 ]
[-3.785041   -1.04681397 -2.97746106 -0.44305266 -3.73951336]
[-3.59846419 -1.08089866 -0.22183591 -1.18110758 -1.1934413 ]
[-3.86345309 -1.64929725 -1.15146805 -1.20137871 -1.20207854]
[-1.39182157 -3.33697211 -1.23544622 -2.25861834 -0.19558185]


In [19]:
part1.v_bounds

(-4.0, 0.25)

All the same conclusions apply to `initialize_velocity()`

`update_binary_position()` outsources all of its function to `convert_pos_to_binary()` so as long as I'm sure that it's passing the correct arguments (I am) and that it's assigning the returned value to the correct variable (I am), then I just have to convince myself that `convert_pos_to_binary()` is correct to be confident in `update_binary_position()`.

In [20]:
testx = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
testu = np.random.uniform(size=testx.size)

In [21]:
testx

array([1., 2., 3., 4., 5.])

In [22]:
testu

array([0.36846507, 0.99720522, 0.69820463, 0.44106394, 0.49651695])

In [23]:
testexpx = expit(testx)

In [24]:
testexpx

array([0.73105858, 0.88079708, 0.95257413, 0.98201379, 0.99330715])

I tested `expit()` against [this calculator](https://www.fxsolver.com/solve/) to see if the function behaved like I expected. It did. 

In [25]:
testu < testexpx

array([ True, False,  True,  True,  True])

In [26]:
testu < expit(np.array([-2, -1, 0, 1, 2]))

array([False, False, False,  True,  True])

These give the boolean results I was expecting. I trust `convert_pos_to_binary()` and by extension, I trust `update_binary_position()`.

`update_position()` is a very simple vector addition equation. I visually confirmed that the equation matches the one in the text. I'll verify that ndarray addition works like vector addition.

In [27]:
testa = np.array([1, 2, 3, 4, 5])
testb = np.array([1, 2, 3, 4, 5])
testc = testa + testb
testc

array([ 2,  4,  6,  8, 10])

I'll also verify that the clipping is working like I expect.

In [28]:
testa[testa < 3] = 3

In [29]:
testa

array([3, 3, 3, 4, 5])

In [30]:
testb[testb > 3] = 3

In [31]:
testb

array([1, 2, 3, 3, 3])

In [32]:
part1.x = np.array([5.0, 4.0, 3.0, 2.0, 1.0])
part1.v = np.array([3.0, 3.0, 3.0, 3.0, 3.0])

In [33]:
part1.x

array([5., 4., 3., 2., 1.])

In [34]:
part1.v

array([3., 3., 3., 3., 3.])

In [35]:
part1.x + part1.v

array([8., 7., 6., 5., 4.])

In [36]:
part1.x_bounds

(-6.0, 6.0)

In [37]:
part1.update_position()

In [38]:
part1.x

array([6., 6., 6., 5., 4.])

In [39]:
part1.v

array([3., 3., 3., 3., 3.])

In [40]:
part1.x = (part1.x * -1) + 2

In [41]:
part1.x

array([-4., -4., -4., -3., -2.])

In [42]:
part1.v = part1.v * -1

In [43]:
part1.v

array([-3., -3., -3., -3., -3.])

In [44]:
part1.update_position()

In [45]:
part1.x

array([-6., -6., -6., -6., -5.])

In [46]:
part1.v

array([-3., -3., -3., -3., -3.])

I'm satisfied that the clipping works like I expected it to. I'm going to reinitialize to get rid of all the manual editing I just did to the attributes.

In [47]:
part1 = COMB_Particle(2.1, 2.1, 2.1, 5, (-6.0, 6.0), (-4.0, 0.25), (0.4, 0.9))

`update_velocity()` is much more complicated. I carefully double checked that the equation matched the text. I will check the individual terms now and substitute non random variables so that I can test the math.

In [48]:
part1.w

0

In [49]:
part1.v

array([-1.43464834, -1.21500088, -3.27713762, -3.12516513,  0.09912318])

In [50]:
part1.w * part1.v

array([-0., -0., -0., -0.,  0.])

In [51]:
part1.w = 0.6

In [52]:
part1.w

0.6

In [53]:
part1.v

array([-1.43464834, -1.21500088, -3.27713762, -3.12516513,  0.09912318])

In [54]:
part1.w * part1.v

array([-0.86078901, -0.72900053, -1.96628257, -1.87509908,  0.05947391])

In [55]:
part1.w

0.6

In [56]:
part1.v

array([-1.43464834, -1.21500088, -3.27713762, -3.12516513,  0.09912318])

In [57]:
part1.c1

2.1

In [58]:
part1.c2

2.1

In [59]:
part1.c3

2.1

In [60]:
part1.ndim

5

In [61]:
np.array([1 for i in range(part1.ndim)])

array([1, 1, 1, 1, 1])

In [62]:
part1.pbest

array([ 2.75264903,  2.54902447, -3.13381338,  0.99502314, -0.55662921])

In [63]:
part1.pbest = part1.pbest + 2

In [64]:
part1.pbest

array([ 4.75264903,  4.54902447, -1.13381338,  2.99502314,  1.44337079])

In [65]:
part1.x

array([ 2.75264903,  2.54902447, -3.13381338,  0.99502314, -0.55662921])

In [66]:
testgbest = part1.pbest + 1
testgbest

array([ 5.75264903,  5.54902447, -0.13381338,  3.99502314,  2.44337079])

In [67]:
testabest = part1.pbest + 2
testabest

array([6.75264903, 6.54902447, 0.86618662, 4.99502314, 3.44337079])

In [68]:
(part1.c1*np.array([1 for i in range(part1.ndim)])*(part1.pbest-part1.x))

array([4.2, 4.2, 4.2, 4.2, 4.2])

Looks good. pbest was originally equivalent to x but I incremented it by 2. So this should have been 2.1 * array of 1's size 5 * array of 2's size 5 = array of 4.2's size 5. Check! The next two terms should be similar but 2.1 * 3 = 6.3 and 2.1 * 4 = 8.4

In [69]:
(part1.c2*np.array([1 for i in range(part1.ndim)])*(testgbest-part1.x))

array([6.3, 6.3, 6.3, 6.3, 6.3])

In [70]:
(part1.c3*np.array([1 for i in range(part1.ndim)])*(testabest-part1.x))

array([8.4, 8.4, 8.4, 8.4, 8.4])

Looks great! Now to make sure that they all add up correctly. should end up with an array of size 5 where each element = 18.9 + the elements from w * v. To verify, I'll run the equation, then subtract 18.9 from it.

In [71]:
(
  (part1.w*part1.v)
+ (part1.c1*np.array([1 for i in range(part1.ndim)])*(part1.pbest-part1.x))
+ (part1.c2*np.array([1 for i in range(part1.ndim)])*(testgbest-part1.x))
+ (part1.c3*np.array([1 for i in range(part1.ndim)])*(testabest-part1.x))
) - 18.9

array([-0.86078901, -0.72900053, -1.96628257, -1.87509908,  0.05947391])

In [72]:
part1.w * part1.v

array([-0.86078901, -0.72900053, -1.96628257, -1.87509908,  0.05947391])

Awesome! I trust `update_velocity()`

Now to check the last two methods: `update_inertia()` and its dependency, `jaccard_index()`.

In [73]:
testgbinary = np.array([1, 0, 1, 0, 1]).astype(np.bool_)
testgbinary

array([ True, False,  True, False,  True])

In [74]:
part1.b

array([ True,  True, False,  True,  True])

**NOTE:** These tests will look different to anyone who runs this after me because part1.b is randomly generated.

In [75]:
test2gbinary = np.array([False, True, False, True, False])
test3gbinary = np.array([True, True, False, True, True])

In [76]:
part1.b

array([ True,  True, False,  True,  True])

In [77]:
testgbinary

array([ True, False,  True, False,  True])

In [78]:
test2gbinary

array([False,  True, False,  True, False])

In [79]:
test3gbinary

array([ True,  True, False,  True,  True])

The jaccard index for testgbinary should be M11 = 0, n = 5, M00 = 0; J = 0/5 = 0

The jaccard index for test2gbinary should be M11 = 2, n = 5, M00 = 3; J = 2/2 = 1

The jaccard index for test3gbinary should be M11 = 2, n = 5, M00 = 1; J = 1/4 = 0.50

In [80]:
part1.jaccard_index(testgbinary)

0.4

0.0

In [81]:
part1.jaccard_index(test2gbinary)

0.5

In [82]:
part1.jaccard_index(test3gbinary)

1.0

0.0

Uh oh, two of them gave back 0.0 and one divided by 0 which should be impossible. Something is wrong. Better test this line by line.

In [83]:
m11, m00 = 0, 0

In [84]:
m11

0

In [85]:
m00

0

In [86]:
part1.b

array([ True,  True, False,  True,  True])

In [87]:
part1.ndim

5

In [88]:
test2gbinary

array([False,  True, False,  True, False])

In [89]:
for i in range(part1.ndim):
    print(i)

0
1
2
3
4


In [90]:
test2gbinary[0] is part1.b[0]

False

In [91]:
test2gbinary[0] is True

False

In [92]:
print('b            : {}'.format(str(part1.b)))
print('test2gbinary : {}'.format(str(test2gbinary)))
m11, m00 = 0, 0
print('\nm11 = {}\nm00 = {}\n'.format(m11, m00))
for i in range(part1.ndim):
    print('Counter: i = {}'.format(i))
    if test2gbinary[i] is part1.b[i]:
        print('IF Statement ONE')
        if test2gbinary[i] is True:
            print('IF Statement TWO-True')
            m11 += 1
        else:
            print('IF Statement TWO-Else')
            m00 += 1
    print('\nm11 = {}\nm00 = {}'.format(m11, m00))
    print('return here would be: {}\n'.format(m11/(part1.ndim-m00)))

b            : [ True  True False  True  True]
test2gbinary : [False  True False  True False]

m11 = 0
m00 = 0

Counter: i = 0

m11 = 0
m00 = 0
return here would be: 0.0

Counter: i = 1
IF Statement ONE
IF Statement TWO-Else

m11 = 0
m00 = 1
return here would be: 0.0

Counter: i = 2
IF Statement ONE
IF Statement TWO-Else

m11 = 0
m00 = 2
return here would be: 0.0

Counter: i = 3
IF Statement ONE
IF Statement TWO-Else

m11 = 0
m00 = 3
return here would be: 0.0

Counter: i = 4

m11 = 0
m00 = 3
return here would be: 0.0



In [93]:
test2gbinary[1]

True

In [94]:
test2gbinary[1] is True

False

In [95]:
test2gbinary[1] == True

True

WHOA. An element of an boolean ndarray that == True does not is True

In [96]:
print('b            : {}'.format(str(part1.b)))
print('test2gbinary : {}'.format(str(test2gbinary)))
m11, m00 = 0, 0
print('\nm11 = {}\nm00 = {}\n'.format(m11, m00))
for i in range(part1.ndim):
    print('Counter: i = {}'.format(i))
    if test2gbinary[i] == part1.b[i]:
        print('IF Statement ONE')
        if test2gbinary[i] == True:
            print('IF Statement TWO-True')
            m11 += 1
        else:
            print('IF Statement TWO-Else')
            m00 += 1
    print('\nm11 = {}\nm00 = {}'.format(m11, m00))
    print('return here would be: {}\n'.format(m11/(part1.ndim-m00)))

b            : [ True  True False  True  True]
test2gbinary : [False  True False  True False]

m11 = 0
m00 = 0

Counter: i = 0

m11 = 0
m00 = 0
return here would be: 0.0

Counter: i = 1
IF Statement ONE
IF Statement TWO-True

m11 = 1
m00 = 0
return here would be: 0.2

Counter: i = 2
IF Statement ONE
IF Statement TWO-Else

m11 = 1
m00 = 1
return here would be: 0.25

Counter: i = 3
IF Statement ONE
IF Statement TWO-True

m11 = 2
m00 = 1
return here would be: 0.5

Counter: i = 4

m11 = 2
m00 = 1
return here would be: 0.5



That seems to have fixed it. I changed all the "is" statements to "==" operators

In [97]:
part1.b

array([ True,  True, False,  True,  True])

In [99]:
testgbinary = np.array([False, False, True, False, False])
test2gbinary = np.array([True, True, False, True, True])
test3gbinary = np.array([False, True, False, True, False])

The jaccard index for testgbinary should be M11 = 0, n = 5, M00 = 0; J = 0/5 = 0

The jaccard index for test2gbinary should be M11 = 4, n = 5, M00 = 1; J = 4/4 = 1

The jaccard index for test3gbinary should be M11 = 2, n = 5, M00 = 1; J = 1/4 = 0.50

In [100]:
print(part1.jaccard_index(testgbinary))
print(part1.jaccard_index(test2gbinary))
print(part1.jaccard_index(test3gbinary))

0.0
1.0
0.5


Yay! It worked! I trust `jaccard_index` with everything except catching a zero division error, but I don't think that is extremely important right now because if `jaccard_index()` is getting passed a gbinary of all `False` then I have problems elsewhere.

In [102]:
part1.w_bounds

(0.4, 0.9)

In [103]:
part1.w

0.6

If I pass the three test gbinary arrays above, update_inertia should give back:
* 0.9-(0.5 * 0) = 0.9
* 0.9-(0.5 * 1) = 0.4
* 0.9-(0.5 * 0.5) = 0.65

These should be the upper and lower bounds, and the halfway point.

In [105]:
print(part1.w)
part1.update_inertia(testgbinary)
print(part1.w)
part1.update_inertia(test2gbinary)
print(part1.w)
part1.update_inertia(test3gbinary)
print(part1.w)

0.65
0.9
0.4
0.65


Yay! I trust `update_inertia()`. I think that's everything. I checked every method and they all did what I believed them to do. I also made sure that the `__init__()` method was doing what I expected. I need to double check when I test `COMB_Swarm` that it is initializing the `COMB_Particle.w` and the `COMB_Particle.p_fitness` attributes correctly.